In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

In [4]:
model = GenerativeModel("gemini-2.0-flash-001")

In [5]:
creative_gen_config = GenerationConfig(temperature=1, top_p=0.8)

In [6]:
prompt = "What are some strategies for overcoming writer's block?"

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

Writer's block is a frustrating experience, but thankfully, there are many strategies to overcome it. Here's a breakdown of effective techniques, categorized for easier understanding:

**I. Shifting Your Mindset:**

*   **Lower the Stakes:** Stop aiming for perfection right away. Remind yourself that the first draft is just that – a draft. It's okay to be messy and imperfect. Focus on getting something down on paper.
*   **Embrace Imperfection:**  "Done is better than perfect."  Don't let the fear of writing something bad stop you from writing anything at all. You can always edit and revise later.
*   **Reframe the Task:** Instead of "writing," think of it as "brainstorming," "exploring," or "talking to yourself on paper."  This can make the process feel less daunting.
*   **Identify the Fear:** What are you afraid of? Is it failure? Criticism? Not being good enough?  Naming the fear can help you address it.
*   **Give Yourself Permission to Write Badly:** Seriously.  Write the worst p

In [7]:
prompt = "Provide ten interview questions for the role of prompt engineer."

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

Okay, here are ten interview questions for a Prompt Engineer role, covering a range of skills and knowledge required for the position:

**Technical Proficiency & Understanding:**

1.  **Describe your experience with different Large Language Models (LLMs) like GPT-3, LaMDA, or others. What are the strengths and weaknesses of each in your opinion, and how does that impact your prompt engineering strategy?**
    *   *Why this question?*  This assesses the candidate's hands-on experience and understanding of the nuances of different models, which is crucial for tailoring prompts effectively. It also gauges their critical thinking.

2.  **Explain the concept of "prompt engineering." What are the key techniques you use to craft effective prompts, and can you provide examples of when you would choose one technique over another (e.g., few-shot learning, chain-of-thought prompting, role-playing)?**
    *   *Why this question?*  This tests the candidate's foundational knowledge of prompt enginee

In [8]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [9]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

Making a reservation.



In [10]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

entertainment



In [11]:
import pandas as pd
from sklearn.metrics import classification_report

review_data_df = pd.DataFrame(
    {
        "review": [
            "i love this product. it does have everything i am looking for!",
            "all i can say is that you will be happy after buying this product",
            "its way too expensive and not worth the price",
            "i am feeling okay. its neither good nor too bad.",
        ],
        "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
    }
)

review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i...,positive
1,all i can say is that you will be happy after ...,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


In [12]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative". Return only the classification.
                review: {row}
              """
    response = model.generate_content(
        contents=prompt, generation_config=predictable_gen_config
    ).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i...,positive,positive\n
1,all i can say is that you will be happy after ...,positive,positive\n
2,its way too expensive and not worth the price,negative,negative\n
3,i am feeling okay. its neither good nor too bad.,neutral,Neutral\n


In [13]:
report = classification_report(
    review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
)
Markdown(report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

    Neutral
       0.00      0.00      0.00       0.0
    negative       0.00      0.00      0.00       1.0
   negative
       0.00      0.00      0.00       0.0
     neutral       0.00      0.00      0.00       1.0
    positive       0.00      0.00      0.00       2.0
   positive
       0.00      0.00      0.00       0.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0


In [14]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [15]:
prompt = """
Provide a very short summary, no more than three sentences, for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:

"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

Quantum computers are currently limited by high error rates due to qubit sensitivity. Quantum error correction, which encodes information across multiple physical qubits to form a more stable "logical qubit," is necessary to reduce these errors. This approach is crucial for enabling large-scale, useful quantum computations.



In [16]:
prompt = """
Provide a very short summary in four bullet points for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Bullet points:

"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

Here's a four-bullet-point summary:

*   Quantum computers use qubits, which are highly susceptible to errors.
*   Current error rates are too high for running useful quantum algorithms.
*   Quantum error correction, using "logical qubits," is needed to reduce errors.
*   Logical qubits encode information across multiple physical qubits for improved accuracy.


In [17]:
prompt = """
Please generate a summary of the following conversation and at the end summarize the to-do's for the support Agent:

Customer: Hi, I'm Larry, and I received the wrong item.

Support Agent: Hi, Larry. How would you like to see this resolved?

Customer: That's alright. I want to return the item and get a refund, please.

Support Agent: Of course. I can process the refund for you now. Can I have your order number, please?

Customer: It's [ORDER NUMBER].

Support Agent: Thank you. I've processed the refund, and you will receive your money back within 14 days.

Customer: Thank you very much.

Support Agent: You're welcome, Larry. Have a good day!

Summary:
"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

Larry contacted support because he received the wrong item and wants to return it for a refund. The support agent offered to process the refund, obtained the order number, and confirmed the refund was processed, stating it would take up to 14 days to receive the money back.

To-Do's for the Support Agent:

*   **Ensure the refund was actually processed in the system.**
*   **(Optional, but good practice) Follow up with Larry in 14 days to confirm he received the refund.**


In [18]:
order = "We need eight grilled cheese sandwiches. Two with swiss cheese, three with muenster, three with cheddar."

prompt = """
    Break the customer's order into individual items with keys for the following fields:
    - item_name
    - cheese_selection

    Order:
    {order_field}
""".format(order_field=order)

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

```json
[
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  }
]
```



In [19]:
prompt = """Q: Who was President of the United States in 1955?
              Which party did he belong to?
            A:
        """
response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

A: Dwight D. Eisenhower
   Republican Party



In [20]:
context = """
Storage and content policy \n
How durable is my data in Cloud Storage? \n
Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
response = model.generate_content(contents=prompt, generation_config=predictable_gen_config)
print(response.text)

[Prompt]
Answer the question given in the contex below:
Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
?

Question: How is high availability achieved? 

Answer:

[R